**Importação dos pacotes necessários:**




In [3]:
import pandas as pd

**Importação dos dados e leitura:**

Essa base de dados está disponivel em: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [4]:
dados = pd.read_csv('/content/creditcard.csv')
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


**Análise breve dos dados de transação:**


In [5]:
print(dados.isna().sum())

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


**Verificando o número de transações e quantas transações são fraudes e a porcentagem delas:**

In [18]:
n_transacoes = dados ['Class'].count()
n_fraudes = dados['Class'].sum()
n_normais = n_transacoes - n_fraudes
fraudes_porc = n_fraudes / n_transacoes
normais_porc = n_normais / n_transacoes


print('Número de transações: ', n_transacoes)
print('Número de fraudes: ', n_fraudes, '%.2f' %(fraudes_porc*100))
print('Número de transações normais: ', n_normais, '%.2f' %(normais_porc*100))


Número de transações:  284807
Número de fraudes:  492 0.17
Número de transações normais:  284315 99.83


**Criação da Árvore de decisão com o scikit-learn:**

In [27]:
from sklearn.model_selection import StratifiedShuffleSplit

def executar_validador(x, y):
  validador = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=0)
  for treino_id, teste_id in validador.split(x, y):
    x_train, x_test = x[treino_id], x[teste_id]
    y_train, y_test = y[treino_id], y[teste_id]
  return x_train, x_test, y_train, y_test
 

**Árvore de Decisão:**

In [33]:
%%time
from sklearn import tree

def executar_classificador(classificador, x_train, x_test, y_train):

    arvore = classificador.fit(x_train, y_train)
    y_pred = arvore.predict(x_test)
    return y_pred

CPU times: user 21 µs, sys: 1 µs, total: 22 µs
Wall time: 28.8 µs


**Visualização dos Dados:**

In [53]:
import matplotlib.pyplot as plt

def salvar_arvore(classificador, nome):
  plt.figure(figsize=(200,100))
  tree.plot_tree(classificador, filled=True, fontsize=14)
  plt.savefig(nome)
  plt.close()


**Medindo a Acurácia da Árvore:**

In [67]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def validar_arvore(y_test, y_pred):
  print(accuracy_score(y_test, y_pred))
  print(precision_score(y_test, y_pred))
  print(recall_score(y_test, y_pred))
  print(confusion_matrix(y_test, y_pred))
  

**Execução do Validador:**

In [39]:
x = dados.drop('Class', axis=1).values
y = dados['Class'].values
x_train, x_test, y_train, y_test = executar_validador(x, y)


**Execução do Classificador DescisionTreeClassifier:**

In [40]:
classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

**Criação da Figura da Árvore de Decisão:**

In [49]:
salvar_arvore(classificador_arvore_decisao, "arvore_decisao1.png")

**Validação da Ávore de Decisão:**

In [68]:
validar_arvore(y_test, y_pred_arvore_decisao)

0.9990168884519505
0.7058823529411765
0.7346938775510204
[[28417    15]
 [   13    36]]


**Avaliação dos Parâmetros:**

In [76]:
print(classificador_arvore_decisao)
print(classificador_arvore_decisao.get_depth())

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')
10


Execução do Classificador DecisionTreeClassifier (**ESSA É A MELHOR DECISÃO**):


In [89]:
%%time
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

CPU times: user 19.5 s, sys: 24.9 ms, total: 19.5 s
Wall time: 19.5 s


In [72]:
validar_arvore(y_test, y_pred_arvore_decisao)

0.9994733330992591
0.9473684210526315
0.7346938775510204
[[28430     2]
 [   13    36]]


Execução do Classificador DecisionTreeClassifier **(ESSA NÃO É A MELHOR DECISÃO):**


In [78]:
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=10, random_state=0, min_samples_leaf=10)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

Validação da Ávore:

In [79]:
validar_arvore(y_test, y_pred_arvore_decisao)

0.9993679997191109
0.8604651162790697
0.7551020408163265
[[28426     6]
 [   12    37]]


Diminuição da Profundidade da Árvore **piorou** o resultado. **(Diminuição para max_depth=5)**

In [80]:
classificador_arvore_decisao = tree.DecisionTreeClassifier(max_depth=5, random_state=0)
y_pred_arvore_decisao = executar_classificador(classificador_arvore_decisao, x_train, x_test, y_train)

In [81]:
validar_arvore(y_test, y_pred_arvore_decisao)

0.999403110845827
0.9210526315789473
0.7142857142857143
[[28429     3]
 [   14    35]]


**Executando o Random Forest com o scikit-learn**




In [87]:
%%time
from sklearn.ensemble import RandomForestClassifier

classificador_random_forest = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=10)
y_pred_random_forest = executar_classificador(classificador_random_forest, x_train, x_test, y_train)

CPU times: user 2min 1s, sys: 130 ms, total: 2min 1s
Wall time: 2min 1s


In [88]:
validar_arvore(y_test, y_pred_random_forest)

0.9995435553526912
0.9736842105263158
0.7551020408163265
[[28431     1]
 [   12    37]]


In [93]:

from sklearn.ensemble import RandomForestClassifier

classificador_random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
y_pred_random_forest = executar_classificador(classificador_random_forest, x_train, x_test, y_train) 

In [94]:
salvar_arvore(classificador_random_forest.estimators_[0], 'reandom_forest1')
salvar_arvore(classificador_random_forest.estimators_[1], 'reandom_forest2')

In [95]:
validar_arvore(y_test, y_pred_random_forest)

0.9995084442259752
0.9487179487179487
0.7551020408163265
[[28430     2]
 [   12    37]]


In [96]:
%%time
from sklearn.ensemble import RandomForestClassifier

classificador_random_forest = RandomForestClassifier(n_estimators=50, random_state=0, max_depth=10)
y_pred_random_forest = executar_classificador(classificador_random_forest, x_train, x_test, y_train) 

CPU times: user 1min 55s, sys: 150 ms, total: 1min 55s
Wall time: 1min 55s


In [97]:
validar_arvore(y_test, y_pred_random_forest)

0.9995435553526912
0.9736842105263158
0.7551020408163265
[[28431     1]
 [   12    37]]
